In [1]:
using MixedModels, DataFrames
using GLM, RData, GeneralizedCopula, Test

datf = joinpath(dirname(pathof(MixedModels)),"..", "test", "dat.rda")
const dat = Dict(Symbol(k)=>v for (k,v) in load(datf));
data = dat[:VerbAgg]

#transform the yes no outcome to 1 0 for logistic regression
out = map(x -> strip(String(x)) == "N" ? 0.0 : 1.0, data[!, :r2])
groups = unique(data[!, :id])
n = length(groups)
d = Bernoulli()
D = typeof(d)
gcs = Vector{GLMCopulaVCObs{Float64, D}}(undef, n)
 for (i, grp) in enumerate(groups)
    gidx = data[!, :id] .== string(grp)
    ni = count(gidx)
    y = Float64.(out[gidx, 1])
    intercept = ones(ni, 1)
    Anger = Float64.(data[gidx, :a])
    X = [intercept Anger]
    V = [ones(ni, ni)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d)
end
gcm_all = GLMCopulaVCModel(gcs);

In [2]:
initialize_model!(gcm_all)
@show gcm_all.β
fill!(gcm_all.Σ, 1.0)
# gcm_all.β .= [-1.0314; 0.0458053] # from the mixed models package
# update σ2 from β using the MM algorithm
update_Σ!(gcm_all)
@show gcm_all.Σ

@test copula_loglikelihood(gcm_all) ==  -4967.48248349655
@test loglikelihood2!(gcm_all, true, true)  == -4967.48248349655

1 0.0 -5303.33883714196 1263
2 -5303.33883714196 -5221.383546700103 315
3 -5221.383546700103 -5221.3445167404 315
4 -5221.3445167404 -5221.344516711747 315
gcm_all.β = [-0.7954736728413148, 0.034971709990645014]
gcm_all.Σ = [0.36107907014459284]


Test Passed

In [3]:
@time GeneralizedCopula.fit2!(gcm_all, IpoptSolver(print_level = 5, derivative_test = "first-order"))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Starting derivative checker for first derivatives.

* grad_f[          1] =  3.6109999974705834e+03    ~ -3.2990074057146608e+03  [ 2.095e+00]
* grad_f[          2] =  7.3771999967107418e+04    ~ -6.7527123610489070e+04  [ 2.092e+00]

Derivative checker detected 2 error(s).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0



 76.494228 seconds (957.08 k allocations: 55.015 MiB, 0.02% gc time)


┌ Warning: Ipopt finished with status Restoration_Failed
└ @ Ipopt /Users/sarahji/.julia/packages/Ipopt/YJcu4/src/MPB_wrapper.jl:178
┌ Warning: Optimization unsuccesful; got Error
└ @ GeneralizedCopula /Users/sarahji/.julia/dev/GeneralizedCopula/src/fit_new.jl:17


GLMCopulaVCModel{Float64,Bernoulli{Float64}}(GLMCopulaVCObs{Float64,Bernoulli{Float64}}[GLMCopulaVCObs{Float64,Bernoulli{Float64}}([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0  …  1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0], [1.0 20.0; 1.0 20.0; … ; 1.0 20.0; 1.0 20.0], [[1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]], [6.422851619622428, 128.45703239244853], [-0.4994240795774004 -9.988481591548009; -0.4994240795774004 -9.988481591548009; … ; -0.4994240795774004 -9.988481591548009; -0.4994240795774004 -9.988481591548009], [5.0e-324], [2.186387517e-314], [-21.975384741953164 -119.72371740563214; -439.5076948390633 -8790.153896781265], [2.1935051213e-314], [-0.9531149757246107, -0.9531149757246107, -0.9531149757246107, -0.9531149757246107, -0.9531149757246107, -0.9531149757246107, -0.9531149757246107, -0.9531149757246107, 1.0491913625003582, 1.0491913625003582  …  1.0491913625003582, 1.0491913625003582, -0.9531149757246107, -0.9531149757246107

In [ ]:
# using the mixed models package we get a higher loglikelihood
Df = DataFrame(y = out, int = ones(length(out)), a = data[:, :a], id = data[:, :id])
verbaggform = @formula(y ~ 1 + a + (1|id));
gm_all = fit(GeneralizedLinearMixedModel, verbaggform, Df, Bernoulli())
@show loglikelihood(gm_all) # -4749.666193096799